In [88]:
# Common imports
import numpy as np
import os


# to make this notebook's output stable across runs
np.random.seed(42)
import pandas as pd
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [89]:
imagesDF = pd.read_csv ("imagesDF.csv")

In [90]:
imagesDF.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,labels
0,0,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,G
1,1,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,U
2,2,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,3
3,3,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,J
4,4,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,E


In [91]:
uniqueLabels = imagesDF['labels'].unique()
uniqueLabels

array(['G', 'U', '3', 'J', 'E', 'F', 'T', 'Z', 'B', 'V', 'R', 'N', '5',
       'K', 'Y', 'L', 'A', 'H', 'S', '9', 'W', 'P', 'C', '2', '4', 'M',
       'X', 'D', '6', '8', 'Q', '7'], dtype=object)

In [92]:
#index, = np.where(y_trainUnique == 'F')
#index
#print ("ord('C') = ", ord('C'))
#print ("chr(67) = ", chr(67))
length = len(uniqueLabels)
for x in range(length):
    label = uniqueLabels[x]
    imagesDF.replace(to_replace={'labels': label} , value=ord(label), inplace = True)

In [93]:
imagesDF.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,labels
0,0,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,71
1,1,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,85
2,2,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,51
3,3,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,74
4,4,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,69


In [94]:
#reference a column 
labelsDF = imagesDF['labels']

In [95]:
labelsDF.head()

0    71
1    85
2    51
3    74
4    69
Name: labels, dtype: int64

In [96]:
#Drop a column
imagesDF.drop(columns =["labels"], inplace = True)
imagesDF.drop(columns =["Unnamed: 0"], inplace = True)

In [97]:
imagesDF.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [98]:
imagesArray = imagesDF.to_numpy()

In [99]:
imagesArray.shape

(39754, 784)

In [100]:
labelsArray = labelsDF.to_numpy()

In [101]:
labelsArray.shape

(39754,)

In [102]:
import tensorflow as tf
tf.__version__

'1.15.5'

In [104]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [124]:
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 91

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int64, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [125]:
#(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(
   imagesArray, labelsArray, test_size=0.2, random_state=11
)

In [126]:
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [127]:
print (np.unique (y_train, return_counts=True))
print (np.unique (y_test, return_counts=True))
np.__version__

(array([50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 74,
       75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]), array([822, 854, 794, 867, 789, 839, 795, 950, 791, 820, 840, 819, 886,
       816, 852, 854, 817, 842, 822, 831, 860, 845, 820, 847, 840, 867,
       849, 796, 849, 840, 836, 854]))
(array([50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 74,
       75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]), array([261, 218, 280, 247, 230, 278, 259, 283, 251, 244, 281, 245, 244,
       236, 250, 234, 234, 236, 238, 237, 257, 259, 254, 267, 257, 221,
       239, 224, 259, 240, 251, 237]))


'1.20.3'

In [128]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [129]:
n_epochs = 10
batch_size = 500

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        
        # Dr. Phil: This inner loop, with shuffle_batch achieves one epoch, 
        # looping over randomly selected batches
        
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            
             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)


0 Last batch accuracy: 0.790099 Test accuracy: 0.79373664
1 Last batch accuracy: 0.9643564 Test accuracy: 0.96038234
2 Last batch accuracy: 0.980198 Test accuracy: 0.9771098
3 Last batch accuracy: 0.97821784 Test accuracy: 0.9800025
4 Last batch accuracy: 0.9841584 Test accuracy: 0.98063135
5 Last batch accuracy: 0.990099 Test accuracy: 0.9825179
6 Last batch accuracy: 0.9841584 Test accuracy: 0.9835241
7 Last batch accuracy: 0.9940594 Test accuracy: 0.9837756
8 Last batch accuracy: 0.9920792 Test accuracy: 0.9839014
9 Last batch accuracy: 0.98613864 Test accuracy: 0.9837756
